In [ ]:
from garminconnect import Garmin
from dotenv import load_dotenv
import os

load_dotenv()

email = os.getenv("GARMIN_EMAIL")
password = os.getenv("GARMIN_PASSWORD")

if not email or not password:
    raise ValueError("Missing credentials - check .env")

client = Garmin(email, password)
client.login()

activities = client.get_activities(0, 200)
#print(activities)
print(type(activities))

In [ ]:
# get activity details using activityId
activity_details = [client.get_activity_details(activity["activityId"]) 
                         for activity in activities if "activityId" in activity]

#print(activity_details)

In [ ]:
import pandas as pd

activities = pd.DataFrame(activities)
activity_details = pd.DataFrame(activity_details)

In [ ]:
merged_activities = pd.merge(activities, activity_details, on="activityId", how="inner")

In [ ]:
#print(merged_activities.columns.tolist())

In [ ]:
columns = [
    "activityId", "activityName", "startTimeLocal", "distance", "duration",
    "averageSpeed", "maxSpeed", "startLatitude", "startLongitude", "calories",
    "averageHR", "maxHR", "aerobicTrainingEffect", "anaerobicTrainingEffect",
    "vO2MaxValue", "activityType", "geoPolylineDTO"
]

df = merged_activities.reindex(columns=columns)
df["startTimeLocal"] = pd.to_datetime(df["startTimeLocal"])
df = df.sort_values(by="startTimeLocal", ascending=False)

df

In [ ]:
import ast
import folium
from folium.plugins import HeatMap

# Containers for different activity types
all_points = []
cycling_points = []
running_points = []

for i, row in df.iterrows():
    try:
        # Convert polyline data
        data_str = row['geoPolylineDTO']
        if isinstance(data_str, str):
            data_str = (
                data_str.replace("null", "None")
                .replace("true", "True")
                .replace("false", "False")
            )
            polyline_data = ast.literal_eval(data_str)
        else:
            polyline_data = data_str

        if not polyline_data:
            print(f"Skipping row {i} (no polyline data)")
            continue

        polyline = polyline_data.get("polyline", [])
        if not polyline:
            print(f"Skipping row {i} (empty polyline)")
            continue

        # Extract activity type
        activity_type = row["activityType"]
        if isinstance(activity_type, str):
            activity_type = ast.literal_eval(activity_type)
        type_key = activity_type.get("typeKey", "").lower()

        added_points = 0
        for point in polyline:
            lat = point.get("lat")
            lon = point.get("lon")
            if lat is not None and lon is not None:
                all_points.append([lat, lon])
                if type_key == "cycling":
                    cycling_points.append([lat, lon])
                elif type_key == "running":
                    running_points.append([lat, lon])
                added_points += 1

        if added_points == 0:
            print(f"Skipping row {i} (no valid lat/lon points)")

    except Exception as e:
        print(f"Skipping row {i} due to error: {e}")

# Print stats
print(f"Collected {len(all_points)} total points")
print(f" - Cycling points: {len(cycling_points)}")
print(f" - Running points: {len(running_points)}")

# Create map centered on Copenhagen (no default OSM layer)
m = folium.Map(location=[55.6761, 12.5683], zoom_start=12, tiles=None)

# Tile layer
folium.TileLayer("OpenStreetMap", name="Activity Type").add_to(m)

# Add feature groups (layer toggle)
fg_all = folium.FeatureGroup(name="All Activities").add_to(m)
fg_cycling = folium.FeatureGroup(name="Cycling").add_to(m)
fg_running = folium.FeatureGroup(name="Running").add_to(m)

# Add heatmaps
if all_points:
    HeatMap(all_points, radius=8, blur=15, max_zoom=15).add_to(fg_all)
else:
    print("No overall points found — check parsing logic")

if cycling_points:
    HeatMap(cycling_points, radius=8, blur=15, max_zoom=15).add_to(fg_cycling)
else:
    print("No cycling points found")

if running_points:
    HeatMap(running_points, radius=8, blur=15, max_zoom=15).add_to(fg_running)
else:
    print("No running points found")

# Add layer control
folium.LayerControl(collapsed=False).add_to(m)

# Save map
output_file = "activity_heatmap.html"
m.save(output_file)
print(f"Heatmap saved to {output_file}")

In [ ]:
# Collect all activity type keys
activity_types = []

for i, row in df.iterrows():
    activity_type = row['activityType']
    try:
        if isinstance(activity_type, str):
            activity_type = ast.literal_eval(activity_type)
        type_key = activity_type.get("typeKey")
        if type_key:
            activity_types.append(type_key)
    except Exception as e:
        print(f"Skipping row {i} due to error: {e}")

# Get unique types
unique_types = sorted(set(activity_types))

print("Found activity types:")
for t in unique_types:
    print(" -", t)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ------------------------
# Data Preparation
# ------------------------

# Filter out zero distance
df_nonzero = df[df['distance'] > 0].copy()

# Extract typeKey safely
df_nonzero['typeKey'] = df_nonzero['activityType'].apply(lambda x: x['typeKey'] if isinstance(x, dict) else 'unknown')

# Aggregate distance and heart rate by activity type
agg_type = df_nonzero.groupby('typeKey').agg({
    'distance':'mean',
    'averageHR':'mean',
    'maxHR':'mean',
    'calories':'mean'
}).reset_index()

# VO2 Max development over time (rolling average)
df_vo2 = df_nonzero[df_nonzero['vO2MaxValue'].notnull()].copy()
df_vo2['startTimeLocal'] = pd.to_datetime(df_vo2['startTimeLocal'])
df_vo2.sort_values('startTimeLocal', inplace=True)
df_vo2['vO2Max_avg'] = df_vo2['vO2MaxValue'].rolling(window=3, min_periods=1).mean()

# ------------------------
# Plotting
# ------------------------

# Colors
color_blue = '#1f77b4'
color_red = '#d62728'

# Set up figure
fig, axes = plt.subplots(3, 2, figsize=(20,18))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# --- 1. Distance per Activity Type with Average HR line ---
agg_type['distance_km'] = agg_type['distance'] / 1000 
ax1 = axes[0,0]
sns.barplot(data=agg_type, x='typeKey', y='distance_km', errorbar=None, ax=ax1, color=color_blue)
ax1.set_ylabel('Average Distance (km)', color=color_blue)
ax1.set_xlabel('')
ax1.tick_params(axis='x', rotation=45)

ax1_twin = ax1.twinx()
sns.lineplot(data=agg_type, x='typeKey', y='averageHR', marker='o', color=color_red, ax=ax1_twin)
ax1_twin.set_ylabel('Average Heart Rate (bpm)', color=color_red)
ax1.set_title("Average Distance & Heart Rate per Activity Type")

# --- 2. Total Calories Burned per Activity Type ---
agg_calories = df_nonzero.groupby('typeKey')['calories'].sum().reset_index()
agg_calories = agg_calories.sort_values('calories', ascending=False) 

sns.barplot(
    data=agg_calories, 
    y='typeKey',  
    x='calories',  
    ax=axes[0,1], 
    color=color_blue, 
    errorbar=None
)
axes[0,1].set_title("Total Calories Burned per Activity Type")
axes[0,1].set_xlabel("Total Calories")
axes[0,1].set_ylabel('')

# --- 3. Calories per Activity Type ---
sns.boxplot(data=df_nonzero, x='typeKey', y='calories', ax=axes[1,0], color=color_blue)
axes[1,0].set_title("Calories Burned per Activity Type")
axes[1,0].set_xlabel('')
axes[1,0].set_ylabel("Calories")
axes[1,0].tick_params(axis='x', rotation=45)

# --- 4. Max HR vs Aerobic Training Effect ---
sns.regplot(data=df_nonzero, x='aerobicTrainingEffect', y='maxHR', ax=axes[1,1],
            scatter_kws={'s':50, 'alpha':0.6}, line_kws={'color':color_red})
axes[1,1].set_title("Max HR vs Aerobic Training Effect")
axes[1,1].set_xlabel("Aerobic Training Effect")
axes[1,1].set_ylabel("Max Heart Rate (bpm)")

# --- 5. Average VO2 Max development over time (rolling average) ---
sns.lineplot(data=df_vo2, x='startTimeLocal', y='vO2Max_avg', marker='o', color=color_blue, ax=axes[2,0])
axes[2,0].set_title("Average VO2 Max Development Over Time")
axes[2,0].set_xlabel("Date")
axes[2,0].set_ylabel("Average VO2 Max Value")
axes[2,0].tick_params(axis='x', rotation=45)

# --- 6. Average Speed vs Distance ---
sns.scatterplot(
    data=df_nonzero,
    x=df_nonzero['distance'] / 1000,  # convert meters to kilometers
    y='averageSpeed',
    hue='typeKey',
    ax=axes[2,1],
    s=50, alpha=0.7
)
axes[2,1].set_title("Average Speed vs Distance by Type")
axes[2,1].set_xlabel("Distance (km)")
axes[2,1].set_ylabel("Average Speed (m/s)")
axes[2,1].legend(title='Type', bbox_to_anchor=(1.05, 1), loc=2)

plt.tight_layout()
plt.show()

#### Regression Line

| Training Effect (x) | Max Heart Rate (y) |
|--------------------|------------------|
| 1                  | 150              |
| 2                  | 155              |
| 3                  | 160              |
| 4                  | 165              |
| 5                  | 170              |

<br><br>

$$
\text{slope} = \frac{y_2 - y_1}{x_2 - x_1} 
= \frac{170 - 150}{5 - 1} 
= \frac{20}{4} 
= 5
$$

$$
\text{intercept} = y_1 - (\text{slope} \cdot x_1) 
= 150 - (5 \cdot 1) 
= 145
$$